In [68]:
import math
import openpyxl


class Phase2:
    """
    Расчитывает параметры тренировок по 2 фазе при подготовке 
    к полумарафону плана Джека Дениелса
    'От 800 метров до марафона' второе издание 
    """
    def __init__(self, week_distance):
        self.week_distance = week_distance # недельный километраж
        self.easy_week_distance = week_distance * 0.25 # max дистанция длительной тренировки
        self.repeats_week_distance = week_distance * 0.05 # max объем дистанций повторов в неделю
        self.threshold_week_distance = week_distance * 0.1 # max объем дистанций в пороговом темпе в неделю
        
    def calculate_distance_long(self, time_long: int, pace_long: str) -> float:
        """
        Расчитывает дистанцию длительной тренировки 25% от
        недельного километража или дистанцмю по заплонированному времени.
        Выводит меньшую дистанцию
        
        Параметры:
        time_long: Время длительного забега по плану в минутах
        pace_long: Темп мин/км. Например '6:45'
        
        Возвращаемое значение:
        float: Дистанцию на длительную тренировку
        """
        # Разделяем темп на минуты и секунды
        pace_minutes, pace_seconds = map(int, pace_long.split(':'))

        # Переводим темп в секунды
        pace_total_seconds = pace_minutes * 60 + pace_seconds

        # Переводим общее время в секунды
        total_time_seconds = time_long * 60

        # Вычисляем расстояние в километрах
        distance_km = total_time_seconds / pace_total_seconds
        result_distance = round(min(distance_km, self.easy_week_distance), 2)

        return result_distance
    
    
    def number_episodes__threshold(self, distance_interval: float) -> int:
        """
        Метод расчитывает количество серий повторов в П-темпе
        
        Параметры:
        distance_interval (float): дистанция интервала в км (1.6, 3.2)
        
        Возвращаемое значение:
        int: Количество серий повторов в П-темпе.
        """
        if self.threshold_week_distance > 6.4:
            distance_threshold = 6.4
        else:
            distance_threshold = self.threshold_week_distance
            
        nm = distance_threshold / distance_interval
        nm = math.floor(nm) # округлить до целого в меньшую сторону    
            
        return nm   
         
    
    
    def number_episodes_repeats(self, plan_repeats: tuple, number_training: int) -> tuple:
        """
        Метод расчитывает количество серий повторов в Пв-темпе
        
        Параметры:
        plan_repeats (tuple): Список повторов в тренировоке. Например ((200, 200), (200, 400))
                             первый элемент в кортеже дистанция в метрах Пв, второй - трусца
        number_training (int): Номер К тренировки.

        Возвращаемое значение:
        int: Количество серий повторов в Пв-темпе.
        
        """
        wd = self.repeats_week_distance
        if number_training == 1:
            wd = wd * .75 # оставить 25% дистанции для второй тренировки
            
        total_repeats = sum(pair[0] for pair in plan_repeats) / 1000
        nm = wd / total_repeats
        nm = math.floor(nm) # округлить до целого в меньшую сторону
        if number_training == 1:
            self.repeats_week_distance = self.repeats_week_distance - total_repeats * nm
        
        return (nm, total_repeats) 
    
    

In [82]:
all_km = 60
pace_long = '6:45'
threshold_long = 1.6
repeats_1_plan = ((200, 200), (200, 400), (400, 200))
repeats_2_plan = ((200, 200),)

p2 = Phase2(all_km) 
dist_long = p2.calculate_distance_long(120, pace_long)
dist_threshold = p2.number_episodes__threshold(threshold_long) * threshold_long

# Первая тренировка в Пв темпе
repeats_1 = p2.number_episodes_repeats(repeats_1_plan, 1)
dist_repeats_1 = repeats_1[0] * repeats_1[1] * 2

# Вторая тренировка в Пв темпе
repeats_2 = p2.number_episodes_repeats(repeats_2_plan, 2)
dist_repeats_2 = (repeats_2[0] * repeats_2[1]) + dist_threshold 

dist_easy = round((all_km - (dist_long + dist_repeats_1 + dist_repeats_2)) / 4, 1)

distance = (dist_easy, dist_repeats_1, dist_easy, dist_easy, dist_repeats_2, dist_easy, dist_long)

wb = openpyxl.open("plan_2phase.xlsx")
ws = wb.worksheets[0]
name_day = ('Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс')
# Заносим дни недели в первый столбец
for i, day in enumerate(name_day, start=1):  # start=1, чтобы начать с первой строки
    ws.cell(row=i, column=1, value=day)  # row=i, column=1 (первый столбец)

for i, d in enumerate(distance, start=1): 
    ws.cell(row=i, column=2, value=d)    

def plan_repeats(plan, repeats):    
    td = ""
    for i in plan:
        td = td + str(i[0]) + " Пв " + str(i[1]) + " трусцой " + " + "    

    return f"{repeats[0]} * ({td[:-4]})"    


Monday = "Л"    
Tuesday = plan_repeats(repeats_1_plan, repeats_1)
Wednesday = "Л"
Thursday = "Л"
Friday = f"{round(dist_threshold, 1)} + {plan_repeats(repeats_2_plan, repeats_2)}"

name_day = (Monday, Tuesday, Wednesday, Thursday, Friday)
for i, w in enumerate(name_day, start=1): 
    ws.cell(row=i, column=3, value=w)    


wb.save("plan_2phase.xlsx")

In [75]:
dist_repeats_2

6.000000000000001

In [76]:
repeats_2

(6, 0.2)

In [78]:
repeats_2

(6, 0.2)

In [79]:
plan_repeats(repeats_2_plan)

'(200, 200) * (200 Пв 200 трусцой)'

In [80]:
repeats_2_plan

((200, 200),)